**Getting Standardization Parameters**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import random_split,Dataset,DataLoader
from torchvision import tv_tensors
from torchvision.transforms import v2
import random
import torch.nn.functional as F
import PIL.Image
import os

In [3]:
os.chdir("/content/drive/MyDrive/DL1008_Final Competition")
data_folder = os.path.join("/content/drive/MyDrive/DL1008_Final Competition/hidden")
print(data_folder)

/content/drive/MyDrive/DL1008_Final Competition/hidden


In [4]:
from unet_model import UNet

**Loading Data**

In [5]:
def split_video_mask(video_folder,index):
    '''split_video_mask returns a tuple of tv_tensors

    :param video_folder: directory to video folder
    :type video_folder: path
    :return: tuple of form (pytorch tensor, pytorch tensor) (size [3, 160, 240],[160, 240])
    '''
    directory = video_folder

    img_path = os.path.join(video_folder,f"image_{index}.png")
    mask_path = os.path.join(video_folder,"mask.npy")

    assert os.path.isfile(img_path) and os.path.isfile(mask_path)

    img = tv_tensors.Image(PIL.Image.open(os.path.join(video_folder,f"image_{index}.png"))).to(torch.get_default_dtype())
    all_masks = torch.from_numpy(np.load(os.path.join(directory,"mask.npy"))).to(torch.get_default_dtype())
    mask = tv_tensors.Image(all_masks[index])

    return img, mask

In [6]:
'''
sample_mean = torch.zeros(3)
sample_var = torch.zeros(3)

for subdir in random.sample(os.listdir(data_folder),60): #about .03 off from true variance
    mask_list = split_video_mask(os.path.join(data_folder,subdir))
    for image,_ in mask_list:
      sample_mean += torch.mean(torch.Tensor(image),dim=[1,2])
      sample_var += torch.var(torch.Tensor(image),dim=[1,2])

    sample_mean = sample_mean/len(mask_list)
    sample_var = sample_var/len(mask_list)

sample_mean = sample_mean/60
sample_std = torch.sqrt(sample_var/60)

mean,std =
#tensor([2.2536, 2.2524, 2.2107]), tensor([1.8866, 1.6751, 2.2449])
'''
sample_mean = torch.tensor([2.2536, 2.2524, 2.2107])
sample_std = torch.tensor([1.8866, 1.6751, 2.2449])

In [7]:
print(sample_mean,sample_std)

tensor([2.2536, 2.2524, 2.2107]) tensor([1.8866, 1.6751, 2.2449])


In [8]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
checkpoint = torch.load("89valjac.pth",map_location=torch.device(device))

In [9]:
best_val_jac = checkpoint["best_val_jac"]
print(best_val_jac.item())

model = UNet(n_channels=3,n_classes=49,bilinear=False)
model.to(device)
model.load_state_dict(checkpoint["model_state_dict"])

88.97635650634766


<All keys matched successfully>

In [10]:
model.eval()
for idx, subdir in enumerate(os.listdir(data_folder)):
    folder_path = os.path.join(data_folder,subdir)

    img_batch = torch.zeros(11,3,160,240)
    for i in range(11):
        img_path = os.path.join(folder_path,f"image_{i}.png")
        img = tv_tensors.Image(PIL.Image.open(img_path)).to(torch.get_default_dtype())
        img_batch[i] = img

    mask = torch.argmax(F.softmax(model(img_batch.to(device)),dim=1),dim=1)
    torch.save(mask,os.path.join(folder_path,"mask.pt"))
    if idx % 100 == 0:
        print(idx)

0
100


KeyboardInterrupt: ignored

Stopped early since this was just a double-check run.